In [2]:
with open("r.txt", 'r') as file:
    text = file.read()
faqs =text

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 5868: character maps to <undefined>

STEP = 1
sentences = [faqs]
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

2024-06-10 16:05:46.293226: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 16:05:47.763766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.fit_on_texts([faqs])

In [5]:
len(tokenizer.word_index)

810

In [6]:
input_sequences = []
for sentence in faqs.split('\n'):
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [7]:
input_sequences

[[49, 221],
 [49, 221, 222],
 [49, 221, 222, 113],
 [49, 221, 222, 113, 223],
 [49, 221, 222, 113, 223, 114],
 [49, 221, 222, 113, 223, 114, 40],
 [160, 372],
 [160, 372, 373],
 [160, 372, 373, 114],
 [160, 372, 373, 114, 40],
 [160, 372, 373, 114, 40, 70],
 [160, 372, 373, 114, 40, 70, 25],
 [160, 372, 373, 114, 40, 70, 25, 1],
 [160, 372, 373, 114, 40, 70, 25, 1, 29],
 [14, 1],
 [14, 1, 5],
 [14, 1, 5, 374],
 [14, 1, 5, 374, 16],
 [14, 1, 5, 374, 16, 24],
 [7, 10],
 [7, 10, 6],
 [7, 10, 6, 115],
 [7, 10, 6, 115, 26],
 [7, 10, 6, 115, 26, 4],
 [7, 10, 6, 115, 26, 4, 375],
 [7, 10, 6, 115, 26, 4, 375, 161],
 [71, 1],
 [71, 1, 30],
 [71, 1, 30, 376],
 [71, 1, 30, 376, 13],
 [71, 1, 30, 376, 13, 116],
 [71, 1, 30, 376, 13, 116, 33],
 [71, 1, 30, 376, 13, 116, 33, 2],
 [71, 1, 30, 376, 13, 116, 33, 2, 24],
 [15, 117],
 [15, 117, 224],
 [15, 117, 224, 225],
 [15, 117, 224, 225, 162],
 [15, 117, 224, 225, 162, 118],
 [7, 87],
 [7, 87, 377],
 [7, 87, 377, 1],
 [7, 87, 377, 1, 71],
 [7, 87, 3

In [8]:
max_len = max([len(x) for x in input_sequences])
max_len

19

In [9]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [10]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,  49, 221],
       [  0,   0,   0, ...,  49, 221, 222],
       [  0,   0,   0, ..., 221, 222, 113],
       ...,
       [  0,   0,   0, ..., 107, 108,   5],
       [  0,   0,   0, ..., 108,   5,  52],
       [  0,   0,   0, ...,   5,  52,  36]], dtype=int32)

In [11]:
X = padded_input_sequences[:,:-1]

In [12]:
y = padded_input_sequences[:,-1]

In [13]:
X.shape

(2635, 18)

In [14]:
y.shape

(2635,)

In [15]:
l = y.shape[0]
l

2635

In [16]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=l+1)

In [17]:
y.shape

(2635, 2636)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [19]:
model = Sequential()
model.add(Embedding(l+1, 100, input_length=19))
model.add(LSTM(150))
model.add(Dense(l+1, activation='softmax'))

/home/mukul-barhate/myenv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
model.fit(X,y,epochs=100)

Epoch 1/100


2024-06-10 16:06:08.149745: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27783440 exceeds 10% of free system memory.


83/83 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.0243 - loss: 7.2284
Epoch 2/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.0460 - loss: 5.8786
Epoch 3/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.0517 - loss: 5.6642
Epoch 4/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.0486 - loss: 5.5537
Epoch 5/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.0552 - loss: 5.4430
Epoch 6/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.0718 - loss: 5.3080
Epoch 7/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.0899 - loss: 5.1619
Epoch 8/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.0856 - loss: 5.1122
Epoch 9/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.1018 - loss: 4.9723
Epoch 10/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.1138 - loss: 4.8665
Epoch 11/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.1103 - loss: 4.7896
Epoch 12/100
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy:

In [23]:
# # Define the prediction function
# def predict_next_words(model, tokenizer, text, num_words=3):
#     for _ in range(num_words):
#         sequence = tokenizer.texts_to_sequences([text])[0]
#         sequence = pad_sequences([sequence], maxlen=19-1, padding='pre')
#         predicted = np.argmax(model.predict(sequence), axis=-1)
#         output_word = ""
#         for word, index in tokenizer.word_index.items():
#             if index == predicted:
#                 output_word = word
#                 break
#         text += " " + output_word
#     return text

In [24]:
# predict_next_words(model, tokenizer,"how are")

NameError: name 'np' is not defined

In [25]:
import numpy as np

In [26]:
def predicted_words(model, tokenizer, text, num_words=3):
    for _ in range(num_words):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=max_len - 1, padding='pre')
        predictions = model.predict(sequence, verbose=0)[0]
        top_3_indices = np.argsort(predictions)[-3:][::-1]
        output_words = [word for word, index in tokenizer.word_index.items() if index in top_3_indices]
        
        print(f"Current text: '{text}'")
        print("Next word choices:")
        for i, word in enumerate(output_words, 1):
            print(f"  Choice {i}: {text} {word}")
        print()
        
        # Here we choose the first word from the top 3 for the next iteration (for simplicity)
        text += " " + output_words[0]
    return text

In [27]:
predicted_words(model, tokenizer , "how are")

Current text: 'how are'
Next word choices:
  Choice 1: how are here
  Choice 2: how are i've
  Choice 3: how are not

Current text: 'how are here'
Next word choices:
  Choice 1: how are here to
  Choice 2: how are here for
  Choice 3: how are here you've

Current text: 'how are here to'
Next word choices:
  Choice 1: how are here to the
  Choice 2: how are here to see
  Choice 3: how are here to assist



'how are here to the'

In [28]:
tokenizer.word_index

{'you': 1,
 'to': 2,
 'we': 3,
 'the': 4,
 'for': 5,
 'a': 6,
 'i': 7,
 'and': 8,
 'of': 9,
 'have': 10,
 'our': 11,
 'your': 12,
 'this': 13,
 'thank': 14,
 "i'm": 15,
 'with': 16,
 'in': 17,
 'is': 18,
 "let's": 19,
 'it': 20,
 'on': 21,
 'will': 22,
 'can': 23,
 'me': 24,
 'are': 25,
 'about': 26,
 'any': 27,
 'that': 28,
 'today': 29,
 'please': 30,
 'do': 31,
 'my': 32,
 'topic': 33,
 'us': 34,
 'or': 35,
 'here': 36,
 'some': 37,
 'welcome': 38,
 "it's": 39,
 'name': 40,
 'been': 41,
 "i've": 42,
 'out': 43,
 'take': 44,
 'course': 45,
 'work': 46,
 'event': 47,
 'key': 48,
 'good': 49,
 'time': 50,
 'what': 51,
 'being': 52,
 'am': 53,
 'one': 54,
 'help': 55,
 'point': 56,
 'field': 57,
 'there': 58,
 'not': 59,
 'discuss': 60,
 'hey': 61,
 'see': 62,
 'new': 63,
 'should': 64,
 'like': 65,
 'an': 66,
 'all': 67,
 'be': 68,
 'questions': 69,
 'how': 70,
 'could': 71,
 'interested': 72,
 'research': 73,
 'academic': 74,
 'success': 75,
 'think': 76,
 'feel': 77,
 'as': 78,
 'so'

In [30]:
# import pickle

In [31]:
# pickle.dump(model,open('modelf.pkl','wb'))